# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
#load the csv
weather_df = "output_data/weather_data.csv"
weather_df = pd.read_csv(weather_df)
weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,0,Ixtapa,20.70,-105.20,80.29,83,90,8.05,1599964578,MX
1,1,Synya,65.37,58.04,42.03,93,100,9.37,1599964579,RU
2,2,Albany,42.60,-73.97,57.22,99,0,8.21,1599964375,US
3,3,Jamestown,42.10,-79.24,64.35,88,90,12.75,1599964579,US
4,4,Clyde River,70.47,-68.59,33.80,83,90,4.70,1599964580,CA


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
gmaps.configure(api_key=g_key)
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
#config gmaps
gmaps.configure(api_key=g_key)

#store the lat and long
locations = weather_df[["Latitude", "Longitude"]]

#store humidity in humidity
humidity = weather_df["Humidity"]

In [23]:
# create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1, opacity=1)
 
fig.add_layer(heat_layer)
plt.savefig("../Images/Vacation_Heat_Map.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
#ideal temp
first = weather_df.loc[weather_df["Max Temp"]<80,:].dropna()
second = first.loc[weather_df["Max Temp"]>70,:].dropna()
second.head()

#wind speed less than 10 mph
third = second.loc[second["Wind Speed"]<10,:].dropna()
third

#zero cloudiness
fourth = third.loc[third["Cloudiness"]==0,:].dropna()
fourth.head()


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
8,8,Mabaruma,8.20,-59.78,77.34,89,0,3.47,1599964581,GY
115,115,Aksu,41.12,80.26,71.15,38,0,3.47,1599964620,CN
135,135,Kiryat Gat,31.61,34.76,73.24,9,0,0.40,1599964628,IL
211,211,Caravelas,-17.71,-39.25,70.97,86,0,6.53,1599964658,BR
262,262,Diamantino,-14.41,-56.45,79.77,25,0,6.42,1599964678,BR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(fourth)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Latitude'] = fifth['Latitude']
hotel_df['Longitude'] = fifth['Longitude']
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel Name
8,Mabaruma,GY,8.20,-59.78,
115,Aksu,CN,41.12,80.26,
135,Kiryat Gat,IL,31.61,34.76,
211,Caravelas,BR,-17.71,-39.25,
262,Diamantino,BR,-14.41,-56.45,


In [8]:
url = "http://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "hotel", "keyword" : "hotel", "radius" : 5000, "key" : g_key}

In [9]:
#loop through rows
for index, row in hotel_df.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    city = row['City']
    country = row['Country']
    
    params["location"] = f"{latitude}, {longitude}"
    
    print(f"Retrieving results for Index {index}: {city}.")
    response = requests.get(url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/results... skipping.")
        
    print("......................")
    
    time.sleep(1)
    
print("-----------------End of Search-------------------")
        

Retrieving results for Index 8: Mabaruma.
Missing field/results... skipping.
......................
Retrieving results for Index 115: Aksu.
Missing field/results... skipping.
......................
Retrieving results for Index 135: Kiryat Gat.
Missing field/results... skipping.
......................
Retrieving results for Index 211: Caravelas.
Missing field/results... skipping.
......................
Retrieving results for Index 262: Diamantino.
Missing field/results... skipping.
......................
Retrieving results for Index 321: Alanya.
Missing field/results... skipping.
......................
Retrieving results for Index 360: Alta Floresta.
Missing field/results... skipping.
......................
Retrieving results for Index 372: Santa Cruz de la Sierra.
Missing field/results... skipping.
......................
Retrieving results for Index 402: Araguari.
Missing field/results... skipping.
......................
Retrieving results for Index 412: Lagoa.
Missing field/results...

In [26]:
#show hotel dataframe
hotel_df


,City,Country,Latitude,Longitude,Hotel Name
8,Mabaruma,GY,8.20,-59.78,
115,Aksu,CN,41.12,80.26,
135,Kiryat Gat,IL,31.61,34.76,
211,Caravelas,BR,-17.71,-39.25,
262,Diamantino,BR,-14.41,-56.45,
321,Alanya,TR,36.54,32.00,
360,Alta Floresta,BR,-9.88,-56.09,
372,Santa Cruz de la Sierra,BO,-17.80,-63.17,
402,Araguari,BR,-18.65,-48.19,
412,Lagoa,PT,39.05,-27.98,


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

#add layer
fig.add_layer(markers)

# Display figure
plt.savefig("../Images/Vacation_Hotel_Map.png")
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>